# Libraries

In [1]:
!pip install surprise

In [2]:
import pandas as pd 
import numpy as np
from numpy import dot
from numpy.linalg import norm 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import Dataset, Reader, accuracy, KNNWithMeans
from surprise.model_selection import GridSearchCV, train_test_split

import random
from random import randint

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

%matplotlib inline

# Data Generation

In [3]:
# Generate data required for the Recommender System
def generateData(n_courses=46, n_topics=5, n_professors=20, n_students=5000, size=20000):
  '''
  This function will generate a dataset with features associated to courses. 
  The dataset will have the following columns:
    - course_id (String): Unique identifier for the course
    - course_topic (Integer): An integer value representing the topic for the course, value is between 1 and 5, 
                              indicating that there are 5 unique topics. Each course can only have 1 topic.
    - professor_id (String): Unique identifier for the professor
    - student_id (String): Unique identifier for the student
    - course_rating (Integer): A value between 0 and 5
  
  params:
    n_courses (Integer): The number of courses in the dataset
    n_topics (Integer): The number of course topics to be chosen from
    n_professors (Integer): Number of professors to be generated
    n_students (Integer): Number of students for the dataset
    size (Integer): The number of rows in the dataset
  
  example:
    data = generateData()
  '''

  d = pd.DataFrame(
      {
          'course_id': [randint(1, n_courses) for _ in range(size)], 
          'course_topic': [randint(1, n_topics) for _ in range(size)],
          'professor_id': [randint(1, n_professors) for _ in range(size)],
          'student_id': [randint(1, n_students) for _ in range(size)],
          'course_ratings': [randint(1, 5) for _ in range(size)]
      }
  ).drop_duplicates()

  return d

random.seed(123)
raw_data = generateData()
raw_data.head(15)

,course_id,course_topic,professor_id,student_id,course_ratings
0,4,5,16,3594,5
1,18,1,20,390,3
2,6,3,6,668,3
3,27,4,11,2633,3
4,18,4,8,3279,1
5,7,5,7,4351,3
6,3,4,3,4812,4
7,25,4,1,3579,3
8,35,1,9,3450,5
9,36,4,11,3869,4


# Recommender System - Collaborative Filtering
## Model Based

In [4]:
# Create the sparse student by course matrix for course ratings
df = raw_data.pivot_table(columns='course_id', index='student_id', values='course_ratings').fillna(0)
mat = df.values
mat = csr_matrix(mat)

df.head()

course_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46
student_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
3,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [5]:
# Normalize predictions
def normalizePrediction(pred):
  return (pred - pred.min()) / (pred.max() - pred.min())

# Generate predictions
def prediction(mat, df, n_factors):
  if not 1 <= n_factors < min(mat.shape):
    raise ValueError('Must be 1 <= n_factors < min(mat.shape)')
  
  # Matrix factorization
  u, s, v = svds(mat, k=n_factors)
  s = np.diag(s)

  # Calculate predictions
  pred = np.dot(np.dot(u, s), v)
  pred = normalizePrediction(pred)

  # Convert it back to dataframe
  pred_df = pd.DataFrame(pred, columns = df.columns, index=list(df.index)).transpose()

  return pred_df

pred_df = prediction(mat, df, 5)

In [6]:
def collaborativeRecommender(pred_df, student_id, top_rec):
  student_pred = pred_df[student_id].reset_index().rename({student_id: 'rating'}, axis=1)
  recommendations = student_pred.sort_values(by='rating', ascending=False).head(top_rec).reset_index(drop=True)

  return recommendations

collaborativeRecommender(pred_df, 1301, 3)

,course_id,rating
0,44,0.445315
1,24,0.439754
2,13,0.434901


# Recommender System - Collaborative Filtering
## Memory Based

### User-User

In [7]:
random.seed(123)
raw_data = generateData()

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(raw_data[['student_id', 'course_id', 'course_ratings']], reader)
trainset, testset = train_test_split(data, test_size=0.3, random_state=123)

In [8]:
model = KNNWithMeans(sim_options={'name': 'cosine', 
                                  'user_based': True})
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [9]:
student = 1301
for course in list(raw_data[(raw_data['student_id']==student)]['course_id']):
  print('predicted course rating for ' + str(course) + ':', model.predict(student, course).est, '\n')
display(raw_data[(raw_data['student_id']==student)])

predicted course rating for 16: 4.677579365079365 

predicted course rating for 27: 5 



,course_id,course_topic,professor_id,student_id,course_ratings
4620,16,1,4,1301,3
15440,27,1,12,1301,5


In [25]:
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)

# train = 80% of the data, holdout = 20% of the data
threshold = int(.8*len(raw_ratings))
train = raw_ratings[:threshold]
holdout = raw_ratings[threshold:]

data.raw_ratings = train 

params = {
    'sim_options': {
        'name': ['msd', 'cosine', 'pearson'], 
        'min_support': [3, 4, 5], 
        'user_based': [True]
        }
    }

cv = GridSearchCV(KNNWithMeans, params, measures=['rmse', 'mae'], cv=5)
cv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [26]:
print('Best parameters:', cv.best_params['rmse'])
print('Best parameters:', cv.best_params['mae'])

print('Best RMSE Score:', cv.best_score['rmse'])
print('Best MAE Score:', cv.best_score['mae'])

Best parameters: {'sim_options': {'name': 'pearson', 'min_support': 3, 'user_based': True}}
Best parameters: {'sim_options': {'name': 'pearson', 'min_support': 3, 'user_based': True}}
Best RMSE Score: 1.771293993437602
Best MAE Score: 1.4491345248712042


In [27]:
model = cv.best_estimator['rmse']

# retrain on the whole set the Training
trainset = data.build_full_trainset()
model.fit(trainset)

# Compute biased accuracy on the Training
predictions = model.test(trainset.build_testset())
print('Biased RMSE on the Training,', end='   ')
accuracy.rmse(predictions)
print('Biased MAE on the Training,', end='   ')
accuracy.mae(predictions)

# Compute unbiased accuracy on the Holdout
testset = data.construct_testset(holdout) 
predictions = model.test(testset)
print('Unbiased RMSE on the Holdout,', end=' ')
accuracy.rmse(predictions)
print('Unbiased MAE on the Holdout,', end=' ')
accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Biased RMSE on the Training,   RMSE: 0.2299
Biased MAE on the Training,   MAE:  0.0568
Unbiased RMSE on the Holdout, RMSE: 1.7357
Unbiased MAE on the Holdout, MAE:  1.3983


1.3983157139200189

### Item-Item

In [22]:
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)

# train = 80% of the data, holdout = 10% of the data
threshold = int(.8*len(raw_ratings))
train = raw_ratings[:threshold]
holdout = raw_ratings[threshold:]

data.raw_ratings = train 

params = {
    'sim_options': {
        'name': ['msd', 'cosine', 'pearson'], 
        'min_support': [3, 4, 5], 
        'user_based': [False]
        }
    }

cv = GridSearchCV(KNNWithMeans, params, measures=['rmse', 'mae'], cv=5)
cv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [23]:
print('Best parameters:', cv.best_params['rmse'])
print('Best parameters:', cv.best_params['mae'])

print('Best RMSE Score:', cv.best_score['rmse'])
print('Best MAE Score:', cv.best_score['mae'])

Best parameters: {'sim_options': {'name': 'pearson', 'min_support': 5, 'user_based': False}}
Best parameters: {'sim_options': {'name': 'pearson', 'min_support': 5, 'user_based': False}}
Best RMSE Score: 1.574069995743117
Best MAE Score: 1.3172781183947435


In [24]:
model = cv.best_estimator['rmse']

# retrain on the whole set the Training
trainset = data.build_full_trainset()
model.fit(trainset)

# Compute biased accuracy on the Training
predictions = model.test(trainset.build_testset())
print('Biased RMSE on the Training,', end='   ')
accuracy.rmse(predictions)
print('Biased MAE on the Training,', end='   ')
accuracy.mae(predictions)

# Compute unbiased accuracy on the Holdout
testset = data.construct_testset(holdout) 
predictions = model.test(testset)
print('Unbiased RMSE on the Holdout,', end=' ')
accuracy.rmse(predictions)
print('Unbiased MAE on the Holdout,', end=' ')
accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Biased RMSE on the Training,   RMSE: 0.3638
Biased MAE on the Training,   MAE:  0.1836
Unbiased RMSE on the Holdout, RMSE: 1.6412
Unbiased MAE on the Holdout, MAE:  1.3426


1.342607713432761

In [16]:
accuracy.mae(predictions)

MAE:  1.3975


1.3975322486748802